# Allostery example

In [ ]:
import allostery

## System setup

A common starting point in MD simulations is a PDB file. `allostery.setup.setup_system()` takes a (clean) input PDB file, and prepares it for further MD simulations. More information is in the description:

In [ ]:
help(allostery.setup.setup_systemup_system)

If a `topology=` parameter is provided, the system will not be re-parameterised. Additionally, if `solvated=True`, no additional preparation will be done, and the function will go straight to minimisation.

An example structure of PTP1B is provided. We will run system setup for 7500 minimisation steps, 100 ps of heating and 250 ps of further equilibration. The file also contains an 11 residue peptide with a phosphorylated tyrosine, and so an additional parameter command is passed.

In [ ]:
equilibrated_system = allostery.setup.setup_system('example_data/input_protein.pdb', 
                                                   (7500, 100, 250),
                                                   parameters=['source leaprc.phosaa10'])

## Run steered MD

Once the system is prepared, the next step is to run steered MD simulations. This allows for better sampling of intermediate conformations which are unstable and therefore short-lived.

In [ ]:
help(allostery.steering.run_smd)

The `topology` and `coordinate` parameters are self-explanatory. The `masks` are AMBER selection masks, corresponding to the atoms involved in each CV used for steering. For example, the distance between the C$\alpha$ atoms of residues 100 and 200 would be ":100@CA :200@CA". More information can be found [here](https://amberhub.chpc.utah.edu/atom-mask-selection-syntax/). `types` corresponds to CV types supported by BioSimSpace.

#### Single steering step

Here is an example of multi-CV steering. The CVs will be the $\chi$1 angle of Tyr152, the distance between C$\gamma$ atoms of residues 196 and 280, and the heavy atom RMSD of residues 178-184. Note that in the masks below, the residue numbers are offset by 1. The system includes an ACE cap at the start, and the mask selection indexes starting from 1. The steering will be carried out in 100 ns. In addition to the specified values, times, and forces, additional steps will be added to apply the force over 4 ps, keeping the CV values as initial. The target values and forces used are based on knowledge of the system.

In [ ]:
steering = allostery.steering.run_smd('system.prm7', 'system_equilibrated.rst7',
                                      masks=[':153&(@N,CA,CB,CG)', ':197@CG :281@CG', ':179-185&!(@/H)'],
                                      types=['torsion', 'distance', 'rmsd'],
                                      timings=[100],
                                      values=[-1.047, 0.7, 0],
                                      forces=[2500, 2500, 2500],
                                      reference='example_data/reference.pdb')

After the steering process is finished, the output files are copied over from the working directory, and additionally a dry copy of the trajectory (no waters or ions) is saved (in case of further analysis required).

We can also check the contents of the PLUMED file written by the `run_sMD()` function:

In [ ]:
with open(f'{steering.workDir()}/plumed.dat', 'r') as file:
    contents = file.readlines()
for line in contents:
    print(line, end='')

Even though we specified only one step of the steered MD, 4 steps in total are part of the plumed file. Between step 0 and 1, the force is applied. Between step 1 and 2, the actual steering happens, and afterwards the simulation is run for another 2 ns with no force.

#### Multiple step steering

In order to specify multiple steering steps, `timings`, `values` and `forces` need to be provided. For example, if we wanted to steer the dihedral angle during the first 50 ns of the simulation and the distance during the second, while steering the RMSD throughout, the input would look like this:

In [ ]:
steering = allostery.steering.run_smd('system.prm7', 'system_equilibrated.rst7',
                                      masks=[':153&(@N,CA,CB,CG)', ':197@CG :281@CG', ':179-185&!(@/H)'],
                                      types=['torsion', 'distance', 'rmsd'],
                                      timings=[50,100],
                                      values=[[-1.047, -1.047,], ['initial', 0.7], ['initial/2', 0]],
                                      forces=[[2500, 2500], [2500, 2500], [2500, 2500]],
                                      reference='example_data/reference.pdb')

Note that simple mathematical operations are allowed for the initial value, and this way the RMSD steering is not affected.

In this case the dihedral angle CV was steered to its target value and kept constant by applying force, while the distance CV was kept at its initial value by applying force during the first half of the simulation. An alternative protocol where they are not steered at all beyond changing the CV value could be employed by simply changing the appropriate force constants to 0.

#### Custom steered MD protocols

`run_smd()` uses BioSimSpace to manage steered MD simulations. For more customization, sMD can be run [directly with BSS](https://github.com/michellab/BioSimSpaceTutorials/tree/main/03_steered_md).

## Analysing steered MD data

Once a steered MD trajectory is produced, it has to be checked to ensure steering has been successful, and snapshots need to be saved for seeded MD simulations. This simple trajectory analysis can be done however the user choses. There is an example notebook in `$ALLOSTERYHOME/data/sMD_analysis.ipynb` which can be a good starting point.

## Seeded MD

With snapshot saved from the sMD trajectory, they can be used as "seeds" to run equilibrium MD simulations. Since they are indeed just equilibrium MD simulations, the `allostery.equilibrium.run_eq_md()` function is most appropriate in this case.

In [ ]:
help(allostery.equilibrium.run_eq_md)

The `coordinates=` parameter would simply be the saved snapshot coordinates.

Please note that this step is highly recommended to be done on a computing cluster with multiple GPU access and running multiple seeded MD simulations in parallel.

## Trajectory featurization

Once seeded MD simulations are finished, they can be used to build a Markov State Model. However, that requires dimensionality reduction, which starts by reducing trajectory data from all atom coordinates to select features.

In [ ]:
help(allostery.analysis.featurize)

`featurize()` computes a distance, dihedral or RMSD values for the trajectory specified, using an AMBER selection mask ([documentation](https://amberhub.chpc.utah.edu/atom-mask-selection-syntax/)). If RMSD is being calculated, `reference` has to be provided as well, and `shared` is the selection mask for atoms used for alignment. For example:

In [ ]:
featurized_trajectory = featurize('examples_data/trajectory.nc', 'distance', ':10@CA :20@CA', topology='example_data/system.prm7')

will calculate the distance between C$\alpha$ atoms for residues 10 and 20. Since the trajectory path was given, topology is also provided. Note that featurization is done by loading the trajectory as `pytraj.Trajectory` which can be slow for larger systems and longer trajectories. If possible, using dry trajectories is recommended.

## In progress

This tool is currently a work in progress. Functionality still to come is:
* MSM building